# Python для анализа данных

## Использование API. Работа с форматами XML

*На основе лекции Ильи Щурова, НИУ ВШЭ*  
*Дополнения: Татьяна Рогович, НИУ ВШЭ*  


# XML

До этого мы с вами собирали данные вручную, обращаясь к html страницам, размеченным для отображения в браузере. Но данные также можно собирать и через API -  — application program interface. Обычный интерфейс — это способ взаимодействия человека с программой, а API — одной программы с другой. Например, вашего скрипта на Python с удалённым веб-сервером. 

Для хранения веб-страниц, которые читают люди, используется язык HTML. Для хранения произвольных структурированных данных, которыми обмениваются между собой программы, используются другие языки — в частности, язык XML, похожий на HTML. Вернее было бы сказать, что XML это метаязык, то есть способ описания языков. В отличие от HTML, набор тегов в XML-документе может быть произвольным (и определяется разработчиком конкретного диалекта XML). Например, если бы мы хотели описать в виде XML некоторую студенческую группу, это могло бы выглядеть так:

```xml
<group>
    <number>134</number>
    <student>
        <firstname>Виталий</firstname>
        <lastname>Иванов</lastname>
    </student>
    <student>
        <firstname>Мария</firstname>
        <lastname>Петрова</lastname>
    </student>
</group>
```

Для обработки XML-файлов можно использовать тот же пакет *Beautiful Soup*, который мы уже использовали для работы с HTML. Единственное различие — нужно указать дополнительный параметр `feautres="xml"` при вызове функции `BeautifulSoup` — чтобы он не искал в документе HTML-теги.

In [1]:
group = """<group>
<number>134</number>
<student>
<firstname>Виталий</firstname>
<lastname>Иванов</lastname>
</student>
<student>
<firstname>Мария</firstname>
<lastname>Петрова</lastname>
</student>
</group>"""

In [2]:
!pip install lxml

In [3]:
from bs4 import BeautifulSoup

obj = BeautifulSoup(group, features="lxml")
print(obj.prettify())

<html>
 <body>
  <group>
   <number>
    134
   </number>
   <student>
    <firstname>
     Виталий
    </firstname>
    <lastname>
     Иванов
    </lastname>
   </student>
   <student>
    <firstname>
     Мария
    </firstname>
    <lastname>
     Петрова
    </lastname>
   </student>
  </group>
 </body>
</html>


Номер группы можно найти, например, вот так - для каждого объекта через точку указываем его атрибут, в который надо спуститься. 

In [4]:
obj.group.number.text # последний атрибут текст, точно также как делали в html

'134'

Но это работает только тогда, когда тэг уникальный. В других случаях, парсер всегда будет попадать в первый child-тэг, который он встретил по пути вниз.

In [5]:
obj.group.student.lastname.text # до Петровой так не добраться

'Иванов'

Перечислить всех студентов можно с помощью цикла (похожая структура у нас была и в обработке html).

In [6]:
for student in obj.group.find_all('student'):
    print(student.lastname.text, student.firstname.text)

Иванов Виталий
Петрова Мария


По сути, главное отличие xml от html, что работать вы будете не со стандартизированными структурами. Поэтому перед работой придется поиграть в детективов - запросить данные и внимательно изучить расположение узлов, чтобы понять, какие тэги вас интересуют.

XML легко представить в виде дерева, где есть главный узел (parent) и его "дети".

![](https://www.py4e.com/images/xml-tree.svg)  
*Источник: Python for Everybody, C.Severance*

Кроме BS парсить xml можно и с помощью других библиотек. Например, ElementTree.


In [7]:
import xml.etree.ElementTree as ET

In [8]:
tree = ET.fromstring(group)
list(tree) # посмотрим, что внутри

[<Element 'number' at 0x00000153C74FD9F0>,
 <Element 'student' at 0x00000153C74FDA40>,
 <Element 'student' at 0x00000153C74FDB30>]

Cинтаксис очень похож на BS. Добрались до первой фамилии.

In [9]:
tree.find('student').find('lastname').text

'Иванов'

In [10]:
for element in tree.findall('student'):
    print(element)
    print(element.find('lastname').text)

<Element 'student' at 0x00000153C74FDA40>
Иванов
<Element 'student' at 0x00000153C74FDB30>
Петрова


Можно немного упростить код, включив дочерний тэг в findall.

In [11]:
for element in tree.findall('student/lastname'):
    print(element.text)

Иванов
Петрова


# Задача
По ссылке данные в формате xml.
http://py4e-data.dr-chuck.net/comments_42.xml

Посчитайте все комментарии в этом документе (поля count).

In [12]:
import requests
data = requests.get('http://py4e-data.dr-chuck.net/comments_42.xml').text

tree = ET.fromstring(data)

total = 0
for element in tree.findall('comments/comment/count'):
#     print(type(element.text)
    total += int(element.text)
    
print(total)

2553


In [13]:
sum([int(element.text) for element in tree.findall('comments/comment/count')])

2553

# Реальный пример: wiki API

Допустим, нам потребовалось получить список всех статей из некоторой категории в Википедии. Мы могли бы открыть эту категорию в браузере и дальше действовать теми методами, которые обсуждались выше. Однако, на наше счастье разработчики Википедии сделали удобное API. Чтобы научиться с ним работать, придётся познакомиться с [документацией](https://www.mediawiki.org/wiki/API:Main_page) (так будет с любым API), но это кажется сложным только в первый раз. Ну хорошо, в первые 10 раз. Или 20. Потом будет проще.

Многие API будут требовать токена (например, ваш google логин-пароль для работы с гугл-документами), но мы сейчас работаем с открытым интерфейсом.

Итак, приступим. Взаимодействие с сервером при помощи API происходит с помощью отправки специальным образом сформированных запросов и получения ответа в одном из машинночитаемых форматов. Нас будет интересовать формат XML, хотя бывают и другие (позже мы познакомимся с JSON). А вот такой запрос мы можем отправить:

https://en.wikipedia.org/w/api.php?action=query&list=categorymembers&cmtitle=Category:Physics&cmsort=timestamp&cmdir=desc&format=xmlfm

Строка `https://en.wikipedia.org/w/api.php` (до знака вопроса) — это *точка входа* в API. Всё, что идёт после знака вопроса — это, собственно, запрос. Он представляет собой что-то вроде словаря и состоит из пар «ключ=значение», разделяемых амперсандом `&`. Некоторые символы приходится кодировать специальным образом.

Например, в адресе выше сказано, что мы хотим сделать запрос (`action=query`), перечислить элементы категории `list=categorymembers`, в качестве категории, которая нас интересует, указана `Category:Physics` (`cmtitle=Category:Physics`) и указаны некоторые другие параметры. Если кликнуть по этой ссылке, откроется примерно такая штука:

```xml
<?xml version="1.0"?>
<api batchcomplete="">
  <continue cmcontinue="2015-05-30 19:37:50|1653925" continue="-||" />
  <query>
    <categorymembers>
      <cm pageid="24293838" ns="0" title="Wigner rotation" />
      <cm pageid="48583145" ns="0" title="Northwest Nuclear Consortium" />
      <cm pageid="48407923" ns="0" title="Hume Feldman" />
      <cm pageid="48249441" ns="0" title="Phase Stretch Transform" />
      <cm pageid="47723069" ns="0" title="Epicatalysis" />
      <cm pageid="2237966" ns="14" title="Category:Surface science" />
      <cm pageid="2143601" ns="14" title="Category:Interaction" />
      <cm pageid="10844347" ns="14" title="Category:Physical systems" />
      <cm pageid="18726608" ns="14" title="Category:Physical quantities" />
      <cm pageid="22688097" ns="0" title="Branches of physics" />
    </categorymembers>
  </query>
</api>
```

Мы видим здесь разные теги, и видим, что нас интересуют теги `<cm>`, находящиеся внутри тега `<categorymembers>`.

Давайте сделаем соответствующий запрос с помощью Python. Для этого нам понадобится уже знакомый модуль `requests`.

In [14]:
url = "https://en.wikipedia.org/w/api.php"
params = {
    'action':'query',
    'list':'categorymembers',
    'cmtitle': 'Category:Physics',
    'format': 'xml'
}

g = requests.get(url, params=params)

Как видно, список параметров мы передаем в виде обычного словаря. Посмотрим, что получилось.

In [15]:
g.ok

True

In [16]:
?g.ok 

Всё хорошо. Теперь используем *Beautiful Soup* для обработки этого XML.

In [17]:
data = BeautifulSoup(g.text, features='xml')

In [18]:
print(data.prettify())

<?xml version="1.0" encoding="utf-8"?>
<api batchcomplete="">
 <continue cmcontinue="subcat|383a4e50464c5a0446340448385a4e3a2e4e011601dc15|1310583" continue="-||"/>
 <query>
  <categorymembers>
   <cm ns="0" pageid="22939" title="Physics"/>
   <cm ns="100" pageid="1653925" title="Portal:Physics"/>
   <cm ns="0" pageid="23479" title="Physicalism"/>
   <cm ns="0" pageid="71771866" title="Six Ideas that Shaped Physics"/>
   <cm ns="14" pageid="36477012" title="Category:Concepts in physics"/>
   <cm ns="14" pageid="49740128" title="Category:Subfields of physics"/>
   <cm ns="14" pageid="694942" title="Category:Physicists"/>
   <cm ns="14" pageid="5625591" title="Category:Physics awards"/>
   <cm ns="14" pageid="70983414" title="Category:Physics by country"/>
   <cm ns="14" pageid="71976587" title="Category:Physics events"/>
  </categorymembers>
 </query>
</api>


Найдём все вхождения тега `<cm>` и выведем их атрибут `title`:

In [19]:
for cm in data.api.query.categorymembers("cm"):
    print(cm['title'])

Physics
Portal:Physics
Physicalism
Six Ideas that Shaped Physics
Category:Concepts in physics
Category:Subfields of physics
Category:Physicists
Category:Physics awards
Category:Physics by country
Category:Physics events


Можно было упростить поиск `<cm>`, не указывая «полный путь» к ним:

In [20]:
for cm in data("cm"):
    print(cm['title'])

Physics
Portal:Physics
Physicalism
Six Ideas that Shaped Physics
Category:Concepts in physics
Category:Subfields of physics
Category:Physicists
Category:Physics awards
Category:Physics by country
Category:Physics events


По умолчанию сервер вернул нам список из 10 элементов. Если мы хотим больше, нужно воспользоваться элементом `continue` — это своего рода гиперссылка на следующие 10 элементов.

In [21]:
data.find("continue")

<continue cmcontinue="subcat|383a4e50464c5a0446340448385a4e3a2e4e011601dc15|1310583" continue="-||"/>

Мне пришлось использовать метод `find()` вместо того, чтобы просто написать `data.continue`, потому что `continue` в Python имеет специальный смысл.

Теперь добавим `cmcontinue` в наш запрос и выполним его ещё раз:

In [22]:
params['cmcontinue'] = data.api("continue")[0]['cmcontinue']

In [23]:
params

{'action': 'query',
 'list': 'categorymembers',
 'cmtitle': 'Category:Physics',
 'format': 'xml',
 'cmcontinue': 'subcat|383a4e50464c5a0446340448385a4e3a2e4e011601dc15|1310583'}

In [24]:
print(data.prettify())

<?xml version="1.0" encoding="utf-8"?>
<api batchcomplete="">
 <continue cmcontinue="subcat|383a4e50464c5a0446340448385a4e3a2e4e011601dc15|1310583" continue="-||"/>
 <query>
  <categorymembers>
   <cm ns="0" pageid="22939" title="Physics"/>
   <cm ns="100" pageid="1653925" title="Portal:Physics"/>
   <cm ns="0" pageid="23479" title="Physicalism"/>
   <cm ns="0" pageid="71771866" title="Six Ideas that Shaped Physics"/>
   <cm ns="14" pageid="36477012" title="Category:Concepts in physics"/>
   <cm ns="14" pageid="49740128" title="Category:Subfields of physics"/>
   <cm ns="14" pageid="694942" title="Category:Physicists"/>
   <cm ns="14" pageid="5625591" title="Category:Physics awards"/>
   <cm ns="14" pageid="70983414" title="Category:Physics by country"/>
   <cm ns="14" pageid="71976587" title="Category:Physics events"/>
  </categorymembers>
 </query>
</api>


In [25]:
g = requests.get(url, params=params)
data = BeautifulSoup(g.text, features='xml')
for cm in data.api.query.categorymembers("cm"):
    print(cm['title'])

Category:History of physics
Category:Physics-related lists
Category:Physics literature
Category:Physical modeling
Category:Physics organizations
Category:Physical systems
Category:Works about physics
Category:Physics stubs


In [26]:
print(data.prettify())

<?xml version="1.0" encoding="utf-8"?>
<api batchcomplete="">
 <query>
  <categorymembers>
   <cm ns="14" pageid="1310583" title="Category:History of physics"/>
   <cm ns="14" pageid="37358141" title="Category:Physics-related lists"/>
   <cm ns="14" pageid="3122431" title="Category:Physics literature"/>
   <cm ns="14" pageid="69843652" title="Category:Physical modeling"/>
   <cm ns="14" pageid="1673597" title="Category:Physics organizations"/>
   <cm ns="14" pageid="10844347" title="Category:Physical systems"/>
   <cm ns="14" pageid="63332556" title="Category:Works about physics"/>
   <cm ns="14" pageid="1064843" title="Category:Physics stubs"/>
  </categorymembers>
 </query>
</api>


Мы получили следующие 10 элементов из категории. Продолжая таким образом, можно выкачать её даже целиком (правда, для этого потребуется много времени).

Аналогичным образом реализована работа с разнообразными другими API, имеющимися на разных сайтах. Где-то API является полностью открытым (как в Википедии), где-то вам потребуется зарегистрироваться и получить application id и какой-нибудь ключ для доступа к API, где-то попросят даже заплатить (например, автоматический поиск в Google стоит что-то вроде 5 долларов за 100 запросов). Есть API, которые позволяют только читать информацию, а бывают и такие, которые позволяют её править. Например, можно написать скрипт, который будет автоматически сохранять какую-то информацию в Google Spreadsheets. Всякий раз при использовании API вам придётся изучить его документацию, но это в любом случае проще, чем обрабатывать HTML-код. Иногда удаётся упростить доступ к API, используя специальные библиотеки.